In [ ]:
import sys
sys.path.insert(0, "/home/581/da1339/AFIM/src")
from obs_seaice_analysis import IceReader

In [18]:
import obs_seaice_analysis.src.IceReader as ice_mod
import importlib
importlib.reload(ice_mod)
from obs_seaice_analysis.src.IceReader import IceReader

In [20]:
r = IceReader(base_dir="/g/data/gv90/da1339/SeaIce/AWI")
sit = r.read_awi(
    var="sea_ice_thickness",
    start_year=2010,
    end_year=2011,
    hemisphere="sh",
    platform="all",
    chunks=None,          # start simple
)
sit.sel(platform='cryosat2')

<xarray.DataArray 'sea_ice_thickness' (time: 24, yc: 216, xc: 216)> Size: 4MB
dask.array<getitem, shape=(24, 216, 216), dtype=float32, chunksize=(1, 216, 216), chunktype=numpy.ndarray>
Coordinates:
  * time      (time) datetime64[ns] 192B 2010-01-01 2010-02-01 ... 2011-12-01
  * xc        (xc) float64 2kB -5.375e+03 -5.325e+03 ... 5.325e+03 5.375e+03
  * yc        (yc) float64 2kB 5.375e+03 5.325e+03 ... -5.325e+03 -5.375e+03
    platform  <U8 32B 'cryosat2'
    lat       (yc, xc) float64 373kB -16.82 -17.22 -17.61 ... -17.22 -16.82
    lon       (yc, xc) float64 373kB -45.0 -44.73 -44.46 ... 135.5 135.3 135.0
Attributes:
    ancillary_variables:    sea_ice_thickness_uncertainty, status_flag, quali...
    comment:                this field is the primary sea ice thickness estim...
    coverage_content_type:  physicalMeasurement
    grid_mapping:           Lambert_Azimuthal_Grid
    long_name:              sea ice thickness
    standard_name:          sea_ice_thickness
    units:                  m

In [30]:
sit = r.read_awi(
    var="sea_ice_thickness",
    start_year=2010,
    end_year=2012,
    hemisphere="sh",
    platforms=["cryosat2", "envisat"],  # cryosat2 preferred over envisat
    collapse_platforms=False,
    chunks="auto",
)
sit

<xarray.DataArray 'sea_ice_thickness' (platform: 2, time: 36, yc: 216, xc: 216)> Size: 13MB
dask.array<concatenate, shape=(2, 36, 216, 216), dtype=float32, chunksize=(1, 1, 216, 216), chunktype=numpy.ndarray>
Coordinates:
  * time      (time) datetime64[ns] 288B 2010-01-01 2010-02-01 ... 2012-12-01
  * xc        (xc) float64 2kB -5.375e+03 -5.325e+03 ... 5.325e+03 5.375e+03
  * yc        (yc) float64 2kB 5.375e+03 5.325e+03 ... -5.325e+03 -5.375e+03
  * platform  (platform) object 16B 'cryosat2' 'envisat'
    lat       (yc, xc) float64 373kB -16.82 -17.22 -17.61 ... -17.22 -16.82
    lon       (yc, xc) float64 373kB -45.0 -44.73 -44.46 ... 135.5 135.3 135.0
Attributes:
    ancillary_variables:    sea_ice_thickness_uncertainty, status_flag, quali...
    comment:                this field is the primary sea ice thickness estim...
    coverage_content_type:  physicalMeasurement
    grid_mapping:           Lambert_Azimuthal_Grid
    long_name:              sea ice thickness
    standard_name:          sea_ice_thickness
    units:                  m

In [33]:
print(sit.sel(platform='cryosat2'))

<xarray.DataArray 'sea_ice_thickness' (time: 36, yc: 216, xc: 216)> Size: 7MB
dask.array<getitem, shape=(36, 216, 216), dtype=float32, chunksize=(1, 216, 216), chunktype=numpy.ndarray>
Coordinates:
  * time      (time) datetime64[ns] 288B 2010-01-01 2010-02-01 ... 2012-12-01
  * xc        (xc) float64 2kB -5.375e+03 -5.325e+03 ... 5.325e+03 5.375e+03
  * yc        (yc) float64 2kB 5.375e+03 5.325e+03 ... -5.325e+03 -5.375e+03
    platform  <U8 32B 'cryosat2'
    lat       (yc, xc) float64 373kB -16.82 -17.22 -17.61 ... -17.22 -16.82
    lon       (yc, xc) float64 373kB -45.0 -44.73 -44.46 ... 135.5 135.3 135.0
Attributes:
    ancillary_variables:    sea_ice_thickness_uncertainty, status_flag, quali...
    comment:                this field is the primary sea ice thickness estim...
    coverage_content_type:  physicalMeasurement
    grid_mapping:           Lambert_Azimuthal_Grid
    long_name:              sea ice thickness
    standard_name:          sea_ice_thickness
    units:       

In [ ]:
import numpy as np
from pathlib import Path
import pygmt

def plot_sit_monthly(
    sit,                         # DataArray (time, yc, xc)
    out_dir,
    vmin=0.0, vmax=5.0,
    cmap="cmocean/matter",
    dpi=300,
    projection="X20c/0",         # Cartesian; width 12 cm, preserve aspect
    frame_x='xaf+lkm',
    frame_y='yaf+lkm',
    platform_name = None,
    show_fig = False
):
    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)

    # Ensure yc increases upward (GMT is happier with ascending coordinates)
    sit = sit.sortby("yc")

    region = [
        float(sit["xc"].min()),
        float(sit["xc"].max()),
        float(sit["yc"].min()),
        float(sit["yc"].max()),
    ]

    for t in sit["time"].values:
        tstr = np.datetime_as_string(t, unit="M")  # "YYYY-MM"
        grid = sit.sel(time=t).load()              # compute only this month
        fig = pygmt.Figure()
        pygmt.makecpt(cmap=cmap, series=[vmin, vmax], reverse=True)
        fig.basemap(
            region=region,
            projection=projection,
            frame=[frame_x, frame_y, f'+tAWI SIT {platform_name} {tstr}'],
        )
        fig.grdimage(grid=grid, cmap=True, nan_transparent=True)
        fig.colorbar(frame=['x+lSea-ice thickness (m)'])

        fig.savefig(out_dir / f"awi_sit_{platform_name}_{tstr}.png", dpi=dpi)
        if show_fig:
            fig.show()

In [37]:
# Example
plot_sit_monthly(sit.sel(platform='cryosat2'), out_dir="/home/581/da1339/graphical/awi_sit_monthly", platform_name='cryosat2')

In [27]:
system("ll ~/graphical/")

['/bin/bash: ll ~/graphical/: No such file or directory']